# Dating Algorithm Recommender System

In [23]:
# libraries
import pandas as pd
import random
import numpy as np

# Creating Data

In [24]:
# Creating a Dataset of men and women
men = pd.DataFrame()

women = pd.DataFrame()

# Number of users
num = 1000

# Dating profile questions for each
qs = ['Q1', 'Q2', 'Q3', 'Q4', 'Q5']

for q in qs:
    men[q] = random.choices(range(10,50,10), k=num)
    
    women[q] = random.choices(range(10,50,10), k=num)
    
    # IDs
    men['id'] = ["m"+str(i) for i in range(num)]
    
    women['id'] = ["w"+str(i) for i in range(num)]
    
# Setting index
men.set_index('id', inplace=True)

women.set_index('id', inplace=True)

In [31]:
# Creating match ratings between users
ratings = pd.DataFrame(index=men.index, columns=women.index)

for i in ratings.columns:
    ratings[i] = random.choices(range(0,6), k=num)
    
ratings

id,w0,w1,w2,w3,w4,w5,w6,w7,w8,w9,...,w990,w991,w992,w993,w994,w995,w996,w997,w998,w999
id,,,,,,,,,,,,,,,,,,,,,
m0,1,0,3,2,4,2,0,3,0,3,...,3,2,3,5,4,2,4,2,3,2
m1,0,2,2,5,0,5,0,1,0,3,...,1,1,3,5,3,1,3,5,4,5
m2,4,3,2,0,0,3,3,0,1,3,...,5,0,5,3,4,4,0,4,3,4
m3,1,0,5,2,5,1,2,3,3,1,...,0,3,1,2,5,5,3,2,1,4
m4,4,3,4,2,2,3,0,4,2,1,...,5,5,0,3,1,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
m995,4,2,1,1,1,3,3,4,0,2,...,1,3,4,5,0,0,3,3,2,1
m996,5,1,2,3,2,5,1,2,5,3,...,1,5,0,1,1,2,5,2,5,1
m997,1,5,5,5,2,1,3,2,0,2,...,0,1,4,2,3,5,5,1,0,5


# Finding Similar Users

### Users who had the most 0 ratings
Meaning they haven't seen the other person yet

In [71]:
# Man
m_user = ratings.T.apply(pd.Series.value_counts).T.sort_values(by=0, ascending=False).iloc[0]

m_user

0    204
1    179
2    158
3    147
4    149
5    163
Name: m111, dtype: int64

In [72]:
# Woman
w_user = ratings.apply(pd.Series.value_counts).T.sort_values(by=0, ascending=False).iloc[0]

w_user

0    206
1    170
2    145
3    157
4    162
5    160
Name: w412, dtype: int64

## Users that haven't been rated/matched yet by the user above

In [105]:
# Man's unrated users
m_nrate = ratings.T[ratings.T[m_user.name]==0].index

m_nrate

Index(['w2', 'w11', 'w13', 'w22', 'w23', 'w30', 'w46', 'w49', 'w55', 'w59',
       ...
       'w950', 'w952', 'w962', 'w966', 'w967', 'w974', 'w979', 'w982', 'w993',
       'w994'],
      dtype='object', name='id', length=204)

In [106]:
# Woman's unrated users
w_nrate = ratings[ratings[w_user.name]==0].index

w_nrate

Index(['m0', 'm4', 'm21', 'm25', 'm30', 'm32', 'm40', 'm41', 'm49', 'm50',
       ...
       'm969', 'm971', 'm972', 'm976', 'm977', 'm987', 'm992', 'm995', 'm996',
       'm997'],
      dtype='object', name='id', length=206)

## Top 10 Similar Users

In [109]:
# Similar men
m_sim = men.T.corrwith(men.T[m_user.name]).sort_values(ascending=False)[1:11]

m_sim

id
m971    1.000000
m372    0.943456
m57     0.943456
m624    0.941742
m595    0.931552
m211    0.930261
m332    0.930261
m272    0.923077
m747    0.923077
m996    0.920737
dtype: float64

In [110]:
# Simliar women
w_sim = women.T.corrwith(women.T[w_user.name]).sort_values(ascending=False)[1:11]

w_sim

id
w97     1.000000
w141    0.943242
w707    0.943242
w753    0.943242
w506    0.943242
w348    0.943242
w450    0.918559
w223    0.918559
w260    0.918559
w382    0.912871
dtype: float64

## Similar Users' Ratings for the Unseen Users

In [122]:
# Similar mens' ratings
msim_rate = ratings.loc[list(m_sim.index)][m_nrate]

msim_rate

id,w2,w11,w13,w22,w23,w30,w46,w49,w55,w59,...,w950,w952,w962,w966,w967,w974,w979,w982,w993,w994
id,,,,,,,,,,,,,,,,,,,,,
m971,3,3,5,1,4,4,5,5,0,1,...,5,1,5,2,1,5,1,0,2,5
m372,1,1,2,1,1,1,5,5,3,1,...,5,2,1,3,2,0,0,1,0,1
m57,3,5,1,1,2,5,5,0,5,5,...,2,0,1,0,1,1,5,2,4,2
m624,2,4,5,4,3,1,5,5,2,5,...,0,0,0,4,3,3,3,0,0,2
m595,0,3,5,5,5,2,5,4,5,0,...,0,3,1,3,3,2,5,2,4,0
m211,5,2,1,0,0,4,3,4,0,3,...,2,4,0,4,5,1,0,0,3,3
m332,0,3,1,5,5,0,2,1,5,0,...,4,0,0,2,0,4,5,0,1,3
m272,4,5,1,1,5,5,5,1,5,1,...,2,0,5,3,3,1,4,0,0,4
m747,4,0,2,4,0,1,2,3,4,2,...,1,2,2,5,2,2,0,5,0,0


In [123]:
# Similar womens' ratings
wsim_rate = ratings[list(w_sim.index)].T[w_nrate]

wsim_rate

id,m0,m4,m21,m25,m30,m32,m40,m41,m49,m50,...,m969,m971,m972,m976,m977,m987,m992,m995,m996,m997
id,,,,,,,,,,,,,,,,,,,,,
w97,0,4,0,3,1,1,3,4,1,0,...,4,3,2,1,5,0,2,2,1,2
w141,1,0,2,1,4,4,5,0,4,1,...,5,3,0,0,3,2,2,4,4,1
w707,0,2,1,0,2,2,2,5,3,3,...,1,1,0,4,1,5,4,3,4,0
w753,3,3,5,4,5,3,5,3,5,4,...,1,1,3,0,0,4,2,3,4,1
w506,0,4,5,1,4,3,1,5,3,5,...,4,4,4,2,1,4,4,0,0,2
w348,3,4,0,0,2,3,3,0,3,1,...,4,3,3,1,5,4,5,2,2,5
w450,3,4,2,3,3,5,2,2,2,0,...,5,3,5,1,0,1,2,5,5,5
w223,1,0,0,0,2,1,2,4,1,0,...,5,1,4,1,4,1,2,3,0,1
w260,5,3,1,3,2,4,4,5,4,1,...,4,4,1,3,2,0,4,4,1,4


## Prdicting a Match Rating

In [146]:
# Man predictions
m_predict = pd.DataFrame()

# Average
m_predict['avg'] = msim_rate.mean().round()

# Frequency
m_predict['freq'] = msim_rate.mode().T[0]

# Median
m_predict['median'] = msim_rate.median().round()

m_predict

,avg,freq,median
id,,,
w2,2.0,0.0,2.0
w11,3.0,3.0,3.0
w13,2.0,1.0,2.0
w22,2.0,1.0,2.0
w23,3.0,5.0,2.0
...,...,...,...
w974,2.0,1.0,2.0
w979,2.0,0.0,2.0
w982,1.0,0.0,0.0


In [147]:
# Woman predictions
w_predict = pd.DataFrame()

# Average
w_predict['avg'] = wsim_rate.mean().round()

# Frequency
w_predict['freq'] = wsim_rate.mode().T[0]

# Median
w_predict['median'] = wsim_rate.median().round()

w_predict

,avg,freq,median
id,,,
m0,2.0,0.0,2.0
m4,3.0,4.0,3.0
m21,2.0,0.0,2.0
m25,2.0,0.0,2.0
m30,3.0,2.0,2.0
...,...,...,...
m987,2.0,0.0,2.0
m992,3.0,2.0,2.0
m995,3.0,3.0,3.0


# Handling a New User

In [163]:
# Dataframe of new user
new_user = pd.DataFrame(
    [random.choices(range(10,50,10), k=5)],
    columns=men.columns,
    index=['m'+str(int(men.index[-1][1:])+1)]
)

m_new_user

,Q1,Q2,Q3,Q4,Q5
m1000,40,10,10,10,30


## Finding the Top 10 Most Similar Users for the New User

In [196]:
new_sim = men.corrwith(m_new_user.iloc[0], axis=1).sort_values(ascending=False)[:10]

new_sim

id
m465    0.988212
m767    0.988212
m576    0.968246
m989    0.968246
m623    0.968246
m888    0.968246
m528    0.968246
m260    0.968246
m768    0.968246
m58     0.968246
dtype: float64

## Getting Recommendations

In [220]:
# Getting the similar users' ratings
new_sim_rate = ratings.T[new_sim.index]

# Finding top recommended based on the median of the similar users' rating
new_sim_rate.median(axis=1).round().sort_values(ascending=False)

id
w726    5.0
w181    5.0
w546    5.0
w285    5.0
w787    5.0
       ... 
w706    0.0
w837    0.0
w812    0.0
w432    0.0
w780    0.0
Length: 1000, dtype: float64